In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input_sp0.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):  # the heads handle the attention - the communication between the nodes (tokens)
    """ one head of self-attention """

    def __init__(self, head_size):  # takes headsize
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)  # k,q,v like above
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))  # tril is not a parameter of the model, a buffer in pytorch lingo
        # (so it does not get picked up by autograd)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])  # pytorch makes this easy for us, passing just a list of heads
        self.proj = nn.Linear(n_embd, n_embd)  # project back into the original pathway
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # just concatinate the results, concat over channel dimension (dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):  # this handles the per-token (node) computation
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # the inner ff layer has 4 times the embedding dimension (to grow the computation on the residual pathway)
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),  # this is the projection layer going back into the original pathway (for the skip-connection) and going down from 4x
            nn.Dropout(dropout),  # added right before the residual pathway comes back (dropout trains subnetworks by randomly shutting off neurons)
        )

    def forward(self, x):
        return self.net(x)  # this is at a per token level: they think on all the attention data they got individually

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head  # watch out that it works out channelwise (e.g. 32 embedding, 4 heads, 8 headsize)
        self.sa = MultiHeadAttention(n_head, head_size)  # the heads running in parallel - the communication
        self.ffwd = FeedFoward(n_embd)  # simple feedforward: at token level, all tokens do this independently: gathered all info from attention, think on it
        # the ff does the computations
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):  # with skip connection, forking off direct gradient pass-through and block adjustments
        x = x + self.sa(self.ln1(x))  # the x + is the skip connection: it lets the gradient pass through (particularly initially), then the blocks adjust
        x = x + self.ffwd(self.ln2(x))  # note that the layer norms are applied before it goes into self-attention or ff, per token
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        # now the network starts to get really deep: suffers from computation problems, two ways to address this: (see Add & Norm in the figure)
        # 1. skip (residual) connections, see https://github.com/KaimingHe/deep-residual-networks (lets gradients pass through and lets blocks adjust them gradually)
        # 2. layer norms = Norm: similar to batch normalization: ensure each neuron has unit Gaussian distribution accross batch dimension
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm (now it is more common to norm before the computations unlike in the trafo paper - called prenorm)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens (idx also has the PE so we need to crop) (not doing this causes a CUDA error due to index out of bounds)
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5058
step 300: train loss 2.4198, val loss 2.4339
step 400: train loss 2.3502, val loss 2.3563
step 500: train loss 2.2965, val loss 2.3128
step 600: train loss 2.2411, val loss 2.2498
step 700: train loss 2.2062, val loss 2.2199
step 800: train loss 2.1640, val loss 2.1870
step 900: train loss 2.1244, val loss 2.1507
step 1000: train loss 2.1035, val loss 2.1302
step 1100: train loss 2.0693, val loss 2.1174
step 1200: train loss 2.0380, val loss 2.0798
step 1300: train loss 2.0248, val loss 2.0646
step 1400: train loss 1.9916, val loss 2.0361
step 1500: train loss 1.9699, val loss 2.0306
step 1600: train loss 1.9635, val loss 2.0488
step 1700: train loss 1.9406, val loss 2.0144
step 1800: train loss 1.9078, val loss 1.9939
step 1900: train loss 1.9095, val loss 1.9888
step 2000: train loss 1.8847, val loss 1.9950
step 2100: train loss 1.

In [4]:
# increase input text file length

with open('input_sp0.txt', 'r', encoding='utf-8') as f:
    text = f.read()

text += ' ' + text

# Writing to file
with open("input_sp1.txt", "w") as f:
    # Writing data to a file
    f.write(text)


In [ ]:
# with double the file length there is barely any overfitting

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16  # how many independent sequences will we process in parallel?
block_size = 32  # what is the maximum context length for predictions?
max_iters = 5000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input_sp1.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):  # the heads handle the attention - the communication between the nodes (tokens)
    """ one head of self-attention """

    def __init__(self, head_size):  # takes headsize
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)  # k,q,v like above
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))  # tril is not a parameter of the model, a buffer in pytorch lingo
        # (so it does not get picked up by autograd)

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x)  # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))  # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x)  # (B,T,C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])  # pytorch makes this easy for us, passing just a list of heads
        self.proj = nn.Linear(n_embd, n_embd)  # project back into the original pathway
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)  # just concatinate the results, concat over channel dimension (dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):  # this handles the per-token (node) computation
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),  # the inner ff layer has 4 times the embedding dimension (to grow the computation on the residual pathway)
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),  # this is the projection layer going back into the original pathway (for the skip-connection) and going down from 4x
            nn.Dropout(dropout),  # added right before the residual pathway comes back (dropout trains subnetworks by randomly shutting off neurons)
        )

    def forward(self, x):
        return self.net(x)  # this is at a per token level: they think on all the attention data they got individually

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head  # watch out that it works out channelwise (e.g. 32 embedding, 4 heads, 8 headsize)
        self.sa = MultiHeadAttention(n_head, head_size)  # the heads running in parallel - the communication
        self.ffwd = FeedFoward(n_embd)  # simple feedforward: at token level, all tokens do this independently: gathered all info from attention, think on it
        # the ff does the computations
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):  # with skip connection, forking off direct gradient pass-through and block adjustments
        x = x + self.sa(self.ln1(x))  # the x + is the skip connection: it lets the gradient pass through (particularly initially), then the blocks adjust
        x = x + self.ffwd(self.ln2(x))  # note that the layer norms are applied before it goes into self-attention or ff, per token
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        # now the network starts to get really deep: suffers from computation problems, two ways to address this: (see Add & Norm in the figure)
        # 1. skip (residual) connections, see https://github.com/KaimingHe/deep-residual-networks (lets gradients pass through and lets blocks adjust them gradually)
        # 2. layer norms = Norm: similar to batch normalization: ensure each neuron has unit Gaussian distribution accross batch dimension
        self.ln_f = nn.LayerNorm(n_embd)  # final layer norm (now it is more common to norm before the computations unlike in the trafo paper - called prenorm)
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx)  # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device))  # (T,C)
        x = tok_emb + pos_emb  # (B,T,C)
        x = self.blocks(x)  # (B,T,C)
        x = self.ln_f(x)  # (B,T,C)
        logits = self.lm_head(x)  # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens (idx also has the PE so we need to crop) (not doing this causes a CUDA error due to index out of bounds)
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))

0.209729 M parameters
step 0: train loss 4.4086, val loss 4.4042
step 100: train loss 2.6523, val loss 2.6555
step 200: train loss 2.4994, val loss 2.5055
step 300: train loss 2.4131, val loss 2.4304
step 400: train loss 2.3452, val loss 2.3614
step 500: train loss 2.3022, val loss 2.3056
step 600: train loss 2.2582, val loss 2.2726
step 700: train loss 2.1969, val loss 2.2117
step 800: train loss 2.1801, val loss 2.1993
step 900: train loss 2.1333, val loss 2.1499
step 1000: train loss 2.1000, val loss 2.1186
step 1100: train loss 2.0730, val loss 2.0943
step 1200: train loss 2.0477, val loss 2.0648
step 1300: train loss 2.0169, val loss 2.0488
step 1400: train loss 1.9951, val loss 2.0256
step 1500: train loss 1.9823, val loss 2.0090
step 1600: train loss 1.9570, val loss 1.9924
step 1700: train loss 1.9489, val loss 1.9794
step 1800: train loss 1.9309, val loss 1.9527
step 1900: train loss 1.9167, val loss 1.9405
step 2000: train loss 1.9054, val loss 1.9344
step 2100: train loss 1.

In [6]:
# byte-pair encoding
# https://ufal.mff.cuni.cz/~helcl/courses/npfl116/ipython/byte_pair_encoding.html

import re, collections

def get_stats(vocab):
    """Compute frequencies of adjacent pairs of symbols."""
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out



In [7]:
from IPython.display import display, Markdown, Latex

train_data = {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}

bpe_codes = {}
bpe_codes_reverse = {}

num_merges = 10

for i in range(num_merges):
    display(Markdown("### Iteration {}".format(i + 1)))
    pairs = get_stats(train_data)
    best = max(pairs, key=pairs.get)
    train_data = merge_vocab(best, train_data)
    
    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best
    
    print("new merge: {}".format(best))
    print("train data: {}".format(train_data))

### Iteration 1

new merge: ('e', 's')
train data: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w es t </w>': 6, 'w i d es t </w>': 3}


### Iteration 2

new merge: ('es', 't')
train data: {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w est </w>': 6, 'w i d est </w>': 3}


### Iteration 3

...

new merge: ('new', 'est</w>')
train data: {'low </w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 9

new merge: ('low', '</w>')
train data: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'w i d est</w>': 3}


### Iteration 10

new merge: ('w', 'i')
train data: {'low</w>': 5, 'low e r </w>': 2, 'newest</w>': 6, 'wi d est</w>': 3}


In [8]:
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs


def encode(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    display(Markdown("__word split into characters:__ <tt>{}</tt>".format(word)))

    pairs = get_pairs(word)    

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        display(Markdown("__Iteration {}:__".format(iteration)))
        
        print("bigrams in the word: {}".format(pairs))
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
        print("candidate for merging: {}".format(bigram))
        if bigram not in bpe_codes:
            display(Markdown("__Candidate not in BPE merges, algorithm stops.__"))
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        print("word after merging: {}".format(word))
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # don't print end-of-word symbols
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)
   
    return word

In [9]:
encode("lowest")

__word split into characters:__ <tt>('l', 'o', 'w', 'e', 's', 't', '</w>')</tt>

__Iteration 1:__

bigrams in the word: {('l', 'o'), ('o', 'w'), ('w', 'e'), ('t', '</w>'), ('e', 's'), ('s', 't')}
candidate for merging: ('e', 's')
word after merging: ('l', 'o', 'w', 'es', 't', '</w>')


__Iteration 2:__

bigrams in the word: {('l', 'o'), ('w', 'es'), ('o', 'w'), ('es', 't'), ('t', '</w>')}
candidate for merging: ('es', 't')
word after merging: ('l', 'o', 'w', 'est', '</w>')


...

bigrams in the word: {('lo', 'w'), ('w', 'est</w>')}
candidate for merging: ('lo', 'w')
word after merging: ('low', 'est</w>')


__Iteration 6:__

bigrams in the word: {('low', 'est</w>')}
candidate for merging: ('low', 'est</w>')


__Candidate not in BPE merges, algorithm stops.__

('low', 'est')

In [11]:
# BPE w/o output
def get_pairs(word):
    """Return set of symbol pairs in a word.
    Word is represented as a tuple of symbols (symbols being variable-length strings).
    """
    pairs = set()
    prev_char = word[0]
    for char in word[1:]:
        pairs.add((prev_char, char))
        prev_char = char
    return pairs
    

def encode_bp(orig):
    """Encode word based on list of BPE merge operations, which are applied consecutively"""

    word = tuple(orig) + ('</w>',)
    pairs = get_pairs(word)    

    if not pairs:
        return orig

    iteration = 0
    while True:
        iteration += 1
        bigram = min(pairs, key = lambda pair: bpe_codes.get(pair, float('inf')))
        if bigram not in bpe_codes:
            break
        first, second = bigram
        new_word = []
        i = 0
        while i < len(word):
            try:
                j = word.index(first, i)
                new_word.extend(word[i:j])
                i = j
            except:
                new_word.extend(word[i:])
                break

            if word[i] == first and i < len(word)-1 and word[i+1] == second:
                new_word.append(first+second)
                i += 2
            else:
                new_word.append(word[i])
                i += 1
        new_word = tuple(new_word)
        word = new_word
        if len(word) == 1:
            break
        else:
            pairs = get_pairs(word)

    # don't print end-of-word symbols
    if word[-1] == '</w>':
        word = word[:-1]
    elif word[-1].endswith('</w>'):
        word = word[:-1] + (word[-1].replace('</w>',''),)
   
    return word
    

In [12]:
encode_bp("lowest")

('low', 'est')

In [64]:
# train BPE with our input text
import re, collections

def get_stats(vocab):
    """Compute frequencies of adjacent pairs of symbols."""
    pairs = collections.defaultdict(int)
    for word, freq in vocab.items():
        symbols = word.split()
        for i in range(len(symbols)-1):
            pairs[symbols[i],symbols[i+1]] += freq
    return pairs

def merge_vocab(pair, v_in):
    v_out = {}
    bigram = re.escape(' '.join(pair))
    p = re.compile(r'(?<!\S)' + bigram + r'(?!\S)')
    for word in v_in:
        w_out = p.sub(''.join(pair), word)
        v_out[w_out] = v_in[word]
    return v_out

with open('inp_testbpe0.txt', 'r', encoding='utf-8') as f:
    text = f.read()

text = text.replace('\n', ' ')
words = sorted(list(set(text.lower().split(' '))))
wordfreq = {w: text.count(w) for w in words}
#print(words)
#print(wordfreq)

#print({(' '.join([el for el in v.lower()]) + ' </w>'):idx for idx,v in enumerate(text.split(' '))})

#train_data = {(' '.join([el for el in v.lower()]) + ' </w>'):idx for idx,v in enumerate(text.split(' '))}
#train_data = {(' '.join([el for el in v.lower()]) + ' </w>'):idx for idx,v in enumerate(words)}
#train_data = {(v + '</w>'):idx for idx,v in enumerate(words)}
#train_data = {'l o w </w>': 5, 'l o w e r </w>': 2, 'n e w e s t </w>': 6, 'w i d e s t </w>': 3}
#train_data = wordfreq
train_data = {(' '.join([el for el in key]) + ' </w>'):val for key,val in wordfreq.items()}
#print(train_data)
#print(get_stats(train_data))

bpe_codes = {}
bpe_codes_reverse = {}

num_merges = 230

for i in range(num_merges):
    #print("### Iteration {}".format(i + 1))
    pairs = get_stats(train_data)
    best = max(pairs, key=pairs.get)
    train_data = merge_vocab(best, train_data)
    
    bpe_codes[best] = i
    bpe_codes_reverse[best[0] + best[1]] = best
    
    #print("new merge: {}".format(best))
    #print("train data: {}".format(train_data))

print(bpe_codes)
print()

for word in words:
    print(encode_bp(word))

{('i', '</w>'): 0, ('e', '</w>'): 1, ('a', '</w>'): 2, ('t', 'h'): 3, ('s', '</w>'): 4, ('t', '</w>'): 5, ('n', '</w>'): 6, ('th', 'e</w>'): 7, ('r', '</w>'): 8, ('i', 's</w>'): 9, ('r', 'e'): 10, (',', '</w>'): 11, ('i', 't</w>'): 12, ('o', 'u'): 13, ('a', 'n</w>'): 14, ('e', 'r'): 15, ('i', 'n</w>'): 16, ('t', 'o'): 17, ('a', 'n'): 18, ('d', '</w>'): 19, ('a', 'r'): 20, ('y', '</w>'): 21, ('e', 'n'): 22, ('e', 's'): 23, ('th', 'e'): 24, ('u', 's</w>'): 25, ('l', 'i'): 26, ('i', 't'): 27, ('o', 'r</w>'): 28, ('v', 'en'): 29, ('to', '</w>'): 30, ('w', 'e</w>'): 31, ('b', 'u'): 32, (':', '</w>'): 33, ('er', 'e</w>'): 34, ('f', 'or</w>'): 35, ('ou', 'r</w>'): 36, ('re', 'li'): 37, ('reli', 'e'): 38, ('relie', 'v'): 39, ('re', 'ven'): 40, ('reven', 'g'): 41, ('s', 'u'): 42, ('the', 'y</w>'): 43, ('l', 'd</w>'): 44, ('an', 'c'): 45, (';', '</w>'): 46, ('e', 'd</w>'): 47, ('f', 'f'): 48, ('ar', 'e</w>'): 49, ('e', 'c'): 50, ('o', 'm'): 51, ('bu', 't</w>'): 52, ('s', ',</w>'): 53, ('g', 'o')